# 一、tokenizer示例

In [1]:
from transformers import AutoTokenizer
import transformers

print('transformers version: ', transformers.__version__)

model_name = "./mini_tokenizer"
tokenizer = AutoTokenizer.from_pretrained(model_name)

text = "这是经过训练的 mini tokenizer。魑魅魍魉"

input_ids = tokenizer.encode(text, return_tensors="pt")
print(input_ids)
print(len(input_ids[0]))
decoded_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)  # 由于 tokenizer_config.json 中的 add_prefix_space=True，因此开头有空格
print(decoded_text)  # 应完全还原原始文本

transformers version:  4.51.1
tensor([[  223,  2530,  2040,  2733,   265,  2979, 11080,  2161,  3781,   821,
          5405,   648,   266,  1040,   242,  9432,  1040,   238,  1040,   234]])
20
 这是经过训练的 mini tokenizer。魑魅魍魉


## 1. 打印当前tokenizer的特殊字符

In [2]:
print("特殊字符:", tokenizer.special_tokens_map)
print("特殊字符ID:", tokenizer.all_special_ids)
print(tokenizer.unk_token_id)

特殊字符: {'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}
特殊字符ID: [1, 2, 0]
0


## 2. Tokenizer的其他基本用法示例

In [3]:
# 分词示例
tokens = tokenizer.tokenize(text)  # 字节级BPE，显示结果可能为乱码
print("分词结果:", tokens)

# 将tokens转回ID
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print("Token IDs:", token_ids)

# 将ID转回tokens
reconstructed_tokens = tokenizer.convert_ids_to_tokens(token_ids)
print("还原后的Tokens:", reconstructed_tokens)

# 将ID转回文本
reconstructed_text = tokenizer.decode(token_ids, skip_special_tokens=False)
print("还原后的文本:", reconstructed_text)

分词结果: ['Ġ', 'è¿Ļæĺ¯', 'ç»ıè¿ĩ', 'è®Ńç»ĥ', 'çļĦ', 'Ġm', 'ini', 'Ġt', 'ok', 'en', 'iz', 'er', 'ãĢĤ', 'éŃ', 'ĳ', 'éŃħ', 'éŃ', 'į', 'éŃ', 'ī']
Token IDs: [223, 2530, 2040, 2733, 265, 2979, 11080, 2161, 3781, 821, 5405, 648, 266, 1040, 242, 9432, 1040, 238, 1040, 234]
还原后的Tokens: ['Ġ', 'è¿Ļæĺ¯', 'ç»ıè¿ĩ', 'è®Ńç»ĥ', 'çļĦ', 'Ġm', 'ini', 'Ġt', 'ok', 'en', 'iz', 'er', 'ãĢĤ', 'éŃ', 'ĳ', 'éŃħ', 'éŃ', 'į', 'éŃ', 'ī']
还原后的文本:  这是经过训练的 mini tokenizer。魑魅魍魉


In [5]:
print(f"vocab size: {len(tokenizer)}")

# 测试一段对话
messages = [
    {"role": "system", "content": "<s>system\n你是一个优秀的聊天机器人，总是给我正确的回应！</s>\n"},
    {"role": "user", "content": '你好'},
    {"role": "assistant", "content": '你好，我是你的助手！'}
]

prompt_messages = messages[:-1] # 只包含 System 和 User
to_be_masked_prompt = tokenizer.apply_chat_template(prompt_messages, tokenize=False, add_generation_prompt=True)
print(to_be_masked_prompt)


# 使用模板进行文本处理
new_prompt = tokenizer.apply_chat_template(messages, tokenize=False)
print(new_prompt)  # 模型将会从<s>assistant后开始生成，到</s>结束
new_prompt = tokenizer.apply_chat_template(messages, tokenize=True)
new_prompt = new_prompt + [0] * 3
print(new_prompt)
prompt_len = len(to_be_masked_prompt)
new_prompt[:prompt_len] = [0] * prompt_len
new_prompt[-3:] = [0] * 3
print(new_prompt)
print(tokenizer.decode(new_prompt, skip_special_tokens=False))

print(tokenizer.decode([1, 201, 223, 2], skip_special_tokens=False))  # 201 是换行，223 是空格

vocab size: 32000
<s>system
你是一个优秀的聊天机器人，总是给我正确的回应！</s>
<s>user
你好</s>
<s>assistant

<s>system
你是一个优秀的聊天机器人，总是给我正确的回应！</s>
<s>user
你好</s>
<s>assistant
你好，我是你的助手！</s>

[1, 2888, 13125, 201, 1986, 2057, 22430, 10825, 540, 8309, 262, 8808, 1053, 1143, 17050, 7606, 2584, 2, 30362, 1, 223, 1290, 648, 201, 1986, 1020, 2, 30362, 1, 223, 4536, 2895, 3245, 201, 1986, 1020, 262, 15565, 11852, 10001, 2584, 2, 30362, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
<unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>
<s>
 </s>


# 二、数据构造示例

In [10]:
import os
import jsonlines
import json
from tqdm import tqdm
from transformers import AutoTokenizer
import numpy as np

seq_monkey_file_path = 'F:/BaiduNetdiskDownload/data/pretrain_data/mobvoi_seq_monkey_general_open_corpus.jsonl'

# 加载训练好的分词器路径
tokenizer = AutoTokenizer.from_pretrained("./mini_tokenizer")
bos_token = tokenizer.bos_token
eos_token = tokenizer.eos_token

## 1. 检查数据格式

In [13]:
def get_total_lines(file_path):
    """
    获取 JSONL 文件的总行数，不忽略错误，保证能够全面统计。
    """
    print('获取文件总行数...')
    with open(file_path, 'rb') as f:  # 使用二进制模式避免编码问题
        return sum(1 for _ in f)
    
def check_jsonl_format(file_path):
    """
    检查 JSONL 文件中的每一行是否是有效的 JSON 格式，带进度显示，并统计所有有问题的行。
    """
    total_lines = get_total_lines(file_path)  # 获取文件总行数
    valid_lines = 0
    invalid_lines = 0

    # 使用逐行读取，捕获 JSON 和编码错误
    with open(file_path, 'rb') as f:  # 使用二进制读取避免编码问题
        # 使用 tqdm 进度条显示检查进度
        for idx, line in tqdm(enumerate(f), total=total_lines, desc="Checking JSONL format"):
            try:
                # 先尝试将每行数据解码为 UTF-8
                decoded_line = line.decode('utf-8')
                # 然后检查是否是有效的 JSON 格式
                obj = jsonlines.Reader([decoded_line]).read()
                valid_lines += 1
            except UnicodeDecodeError as e:
                print(f"Encoding error at line {idx + 1}: {e}")
                invalid_lines += 1
            except jsonlines.InvalidLineError as e:
                print(f"Invalid JSON at line {idx + 1}: {e}")
                invalid_lines += 1

    print(f"检查完成，文件中共有 {valid_lines} 行有效的 JSON 数据，{invalid_lines} 行无效的 JSON 数据。")
    return valid_lines, invalid_lines

def remove_invalid_line(file_path, output_path, invalid_line_num):
    """
    读取文件，跳过指定的无效行，并将结果写入新文件
    """
    with open(file_path, 'rb') as infile, open(output_path, 'wb') as outfile:
        for idx, line in enumerate(infile):
            if idx + 1 != invalid_line_num:  # 跳过无效行
                outfile.write(line)

valid_lines, invalid_lines = check_jsonl_format(seq_monkey_file_path)

获取文件总行数...


Checking JSONL format: 100%|██████████| 13000000/13000000 [01:54<00:00, 113800.46it/s]

检查完成，文件中共有 13000000 行有效的 JSON 数据，0 行无效的 JSON 数据。


## 2. 如何构造预训练数据

每个预训练数据需要构造为如下形式：`<bos> text <eos>`，然后进行拼接：`<bos> text1 <eos> <bos> text2 <eos>`

【思考】：可不可以不加`<bos>`，只用`<eos>`分割不同样本？

建议同时加上`<bos>`和`<eos>`，可能的原因是：`<bos>`标记序列的起始，`<eos>`标记序列的结束；如果加入`<bos>`，一方面，在预训练时，**模型就已经见过这个特殊token的起始含义，在后续的后训练中，能够很好的适应生成任务**；另一方面，拼接后，`<eos>`之后固定为`<bos>`，即模型能够学习到`<eos><bos>`强烈的结构信号，这样即便是产生跨样本注意，当学习到`<eos><bos>`抽象的边界含义时，**会使模型自适应的将前一个样本的注意力权重减小，而只关注本样本的token，因此通常也可不使用样本掩码，避免计算开销**。

---

【例子】：假设有三个样本序列：`abc`,`defgh`,`ijklmn`，那么加入`<bos>`和`<eos>`并拼接，有`＜bos＞abc＜eos＞＜bos＞defgh＜eos＞＜bos＞ijklmn＜eos＞`

假设`max_seq_len=3`，那么对拼接数据直接分块以构造数据集，每个块的长度为`max_seq_len+1`，以方便截取`input`和`target`，则原始数据分块后为：

- 块 1: `<bos> a b c`
- 块 2: `<eos> <bos> d e`
- 块 3: `f g h <eos>`
- 块 4: `<bos> i j k`
- 块 5: `l m n <eos>`

模型的`input`为：

- 块 1: `<bos> a b`
- 块 2: `<eos> <bos> d`
- 块 3: `f g h`
- 块 4: `<bos> i j`
- 块 5: `l m n`

预测下一个token，即`target`为：

- 块 1: `a b c`
- 块 2: `<bos> d e`
- 块 3: `g h <eos>`
- 块 4: `i j k`
- 块 5: `m n <eos>`

但上述直接分块会使得样本的上下文信息缺失。例如样本`defgh`，在块2中，模型只能通过`d`预测`e`，但`e`后该预测什么，模型没有见过；在块3中，模型从`f`预测`g`，但前文信息`de`模型也没见过。因此，可以采用滑动窗口的策略在一定程度上减少上下文信息缺失。例如，如果滑动窗口步长为2，那么构造的原始数据块为：

- 块 1: `<bos> a b c`
- 块 2: `b c <eos> <bos>`
- 块 3: `<eos> <bos> d e`
- 块 4: `d e f g`
- 块 5: `f g h <eos>`
- 块 6: `<bos> i j k`
- 块 7: `j k l m`
- 块 8: `l m n <eos>`

这会使得数据量增加，但能保证信息的连贯性。

结合上述示例，我们将使用滑动窗口的方式来构造数据集，窗口选择为最大长度的10%。

### 1. 处理seq monkey数据

In [11]:
def process_seq_monkey(jsonl_path: str, bin_path: str, buffer_size: int = 1000000, dtype_str: str = 'uint16'):
    """
    读取 jsonl 文件, 提取文本字段, 分词, 并将 token id 保存为二进制文件

    Args:
        jsonl_path (str): 输入的 jsonl 文件路径。
        bin_path (str): 输出的二进制文件路径。
        buffer_size (int): 写入磁盘前在内存中缓冲的 token id 数量。
        dtype_str (str): 保存 token id 的 numpy 数据类型 ('uint16', 'uint32'等), 'uint16' 适用于词汇量 < 65536 的分词器, 如果词汇量更大，请使用 'uint32'。
    """
    # 选择合适的 NumPy 数据类型
    try:
        dtype = np.dtype(dtype_str)
    except TypeError:
        print(f"错误：无效的 dtype_str '{dtype_str}'。请使用 'uint16', 'uint32' 等。")
        return

    print(f"词汇量大小: {len(tokenizer)}")
    if dtype == np.uint16 and len(tokenizer) > 65535:
        print(f"警告：分词器词汇量大小 ({len(tokenizer)}) 可能超过了 'uint16' 的最大值 (65535)。考虑使用 'uint32'。")
    
    token_buffer = []
    total_tokens = 0

    print(f"开始处理文件: {jsonl_path}")
    try:
        # 获取文件总行数用于进度条
        print("获取文件总行数...")
        with open(jsonl_path, 'r', encoding='utf-8') as f_in:
            total_lines = sum(1 for _ in f_in)
            print(f"文件总行数: {total_lines}")

        with open(jsonl_path, 'r', encoding='utf-8') as f_in, open(bin_path, 'wb') as f_out:

            # 使用 tqdm 显示进度
            for line in tqdm(f_in, total=total_lines, desc="Processing lines"):
                try:
                    # 解析 jsonl 行
                    data = json.loads(line.strip())
                    
                    # 提取文本
                    text = data.get('text')
                    # 分词
                    token_ids = tokenizer.encode(bos_token+text+eos_token)
                    # 添加到缓冲区
                    token_buffer.extend(token_ids)
                    
                    # 如果缓冲区达到大小，则写入文件
                    if len(token_buffer) >= buffer_size:
                        array_to_write = np.array(token_buffer[:buffer_size], dtype=dtype)
                        array_to_write.tofile(f_out)
                        total_tokens += len(array_to_write)
                        token_buffer = token_buffer[buffer_size:] # 保留剩余部分

                except json.JSONDecodeError:
                    print(f"警告：无法解析 jsonl 行: {line.strip()}")
                except Exception as e:
                    print(f"处理行时发生意外错误: {e} - 行内容: {line.strip()}")

            # 处理结束后，写入缓冲区中剩余的 token
            if token_buffer:
                array_to_write = np.array(token_buffer, dtype=dtype)
                array_to_write.tofile(f_out)
                total_tokens += len(array_to_write)
                print(f"写入最后 {len(token_buffer)} 个 tokens。")

    except FileNotFoundError:
        print(f"错误：输入文件未找到 {jsonl_path}")
        return
    except Exception as e:
        print(f"处理文件时发生错误: {e}")
        return

    print("-" * 30)
    print(f"处理完成!")
    print(f"总共写入 {total_tokens} 个 Token ID 到 {bin_path}")
    print(f"使用的数据类型: {dtype.name}")
    print("-" * 30)

process_seq_monkey(seq_monkey_file_path, 'F:/BaiduNetdiskDownload/data/pretrain_data/pretrain_data.bin')

词汇量大小: 32000
开始处理文件: F:/BaiduNetdiskDownload/data/pretrain_data/mobvoi_seq_monkey_general_open_corpus.jsonl
获取文件总行数...
文件总行数: 13000000


Processing lines: 100%|██████████| 13000000/13000000 [3:11:48<00:00, 1129.64it/s] 

写入最后 198601 个 tokens。
------------------------------
处理完成!
总共写入 7812198601 个 Token ID 到 F:/BaiduNetdiskDownload/data/pretrain_data/pretrain_data.bin
使用的数据类型: uint16
------------------------------


In [13]:
def count_unk_token_chunked(bin_path, dtype=np.uint16, unk_token_id=0, chunk_size_items=1024*1024*50):
    """
    分块统计二进制文件中的 UNK token 的数量，并显示进度。

    Args:
        bin_path (str): 二进制文件的路径。
        dtype (np.dtype): 文件中数据项的 numpy 数据类型。
        unk_token_id (int): 要计数的 UNK token 的 ID。
        chunk_size_items (int): 每次读取的数据项数量（不是字节数）。
                                默认值约为 100MB (50 * 1024 * 1024 * 2 bytes)。
    """
    try:
        total_size_bytes = os.path.getsize(bin_path)
        item_size = np.dtype(dtype).itemsize
        if item_size == 0:
             print(f"警告：无法确定数据类型 '{dtype}' 的大小。")
             return
        total_items = total_size_bytes // item_size
        print(f"文件路径: {bin_path}")
        print(f"总数据项数: {total_items:,}")
        print(f"数据类型: {dtype}")
        print(f"查找的 Token ID: {unk_token_id}")
        print(f"分块大小（项）: {chunk_size_items}")

        total_unk_count = 0
        processed_items = 0

        with open(bin_path, 'rb') as f, tqdm(total=total_items, unit='item', desc="处理进度") as pbar:
            while True:
                # 从文件读取一个数据块
                data_chunk = np.fromfile(f, dtype=dtype, count=chunk_size_items)

                # 如果没有读取到数据，说明文件已读完
                if data_chunk.size == 0:
                    break

                # 统计当前块中的 UNK token
                chunk_unk_count = np.sum(data_chunk == unk_token_id)
                total_unk_count += chunk_unk_count

                # 更新进度条
                pbar.update(data_chunk.size)
                processed_items += data_chunk.size

        print(f"\n处理完成。")
        print(f"总共找到的 UNK token 数量: {total_unk_count}")
        print(f"总共处理的数据项: {processed_items}")

    except FileNotFoundError:
        print(f"错误：文件未找到 {bin_path}")
    except Exception as e:
        print(f"处理过程中发生错误: {e}")



In [15]:
count_unk_token_chunked('F:/BaiduNetdiskDownload/data/pretrain_data/pretrain_data.bin')

# 查看pretrain_data.bin的数据
token_ids_example = np.fromfile('F:/BaiduNetdiskDownload/data/pretrain_data/pretrain_data.bin', dtype=np.uint16, count=200)
print(tokenizer.decode(token_ids_example.tolist()))

文件路径: F:/BaiduNetdiskDownload/data/pretrain_data/pretrain_data.bin
总数据项数: 7,812,198,601
数据类型: <class 'numpy.uint16'>
查找的 Token ID: 0
分块大小（项）: 52428800


处理进度: 100%|██████████| 7812198601/7812198601 [00:27<00:00, 281220490.12item/s]


处理完成。
总共找到的 UNK token 数量: 0
总共处理的数据项: 7812198601
<s> 在查处虚开增值税专用发票案件中，常常涉及进项留抵税额和税款损失的认定和处理。在计算税款损失时，要不要将进项留抵税额包括在内？
对此，实务中存在意见分歧。
有人主张归并，即计算税款损失时包括进项留抵税额；
有人主张剥离，即计算税款损失时剔除进项留抵税额。分析这个问题，需要确定进项留抵税额与税款损失之间是什么关系。
理清这二者之间的关系，首先需要了解增值税的概念和其抵扣机制。增值税是以商品（货物、服务等）在流转过程中产生的增值额作为计税依据而征收的一种流转税。为避免重复征税，在增值税中存在抵扣链条机制。
一般而言，交易上游企业缴纳的税额，交易下游


### 2. 处理wikipedia数据

In [16]:
def process_wikipedia(json_path: str, bin_path: str, buffer_size: int = 1000000, dtype_str: str = 'uint16'):
    """
    读取 json 文件, 提取文本字段, 分词, 并将 token id 保存为二进制文件

    Args:
        json_path (str): 输入的 jsonl 文件路径。
        bin_path (str): 输出的二进制文件路径。
        buffer_size (int): 写入磁盘前在内存中缓冲的 token id 数量。
        dtype_str (str): 保存 token id 的 numpy 数据类型 ('uint16', 'uint32'等), 'uint16' 适用于词汇量 < 65536 的分词器, 如果词汇量更大，请使用 'uint32'。
    """
    # 选择合适的 NumPy 数据类型
    try:
        dtype = np.dtype(dtype_str)
    except TypeError:
        print(f"错误：无效的 dtype_str '{dtype_str}'。请使用 'uint16', 'uint32' 等。")
        return

    print(f"词汇量大小: {len(tokenizer)}")
    if dtype == np.uint16 and len(tokenizer) > 65535:
        print(f"警告：分词器词汇量大小 ({len(tokenizer)}) 可能超过了 'uint16' 的最大值 (65535)。考虑使用 'uint32'。")
    
    token_buffer = []
    total_tokens = 0

    print(f"开始处理文件: {json_path}")
    # 获取文件总行数用于进度条
    print("获取文件总行数...")
    with open(json_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
        total_lines = len(data)
        print(f"文件总行数: {total_lines}")

    with open(json_path, 'r', encoding='utf-8') as f_in, open(bin_path, 'wb') as f_out:
        total_data = json.load(f_in)
        # 使用 tqdm 显示进度
        for data in tqdm(total_data, total=total_lines, desc="Processing lines"):
            # 提取文本
            text = data.get('completion')
            # 分词
            token_ids = tokenizer.encode(bos_token+text+eos_token)
            # 添加到缓冲区
            token_buffer.extend(token_ids)
            
            # 如果缓冲区达到大小，则写入文件
            if len(token_buffer) >= buffer_size:
                array_to_write = np.array(token_buffer[:buffer_size], dtype=dtype)
                array_to_write.tofile(f_out)
                total_tokens += len(array_to_write)
                token_buffer = token_buffer[buffer_size:] # 保留剩余部分

        # 处理结束后，写入缓冲区中剩余的 token
        if token_buffer:
            array_to_write = np.array(token_buffer, dtype=dtype)
            array_to_write.tofile(f_out)
            total_tokens += len(array_to_write)
            print(f"写入最后 {len(token_buffer)} 个 tokens。")

    print("-" * 30)
    print(f"处理完成!")
    print(f"总共写入 {total_tokens} 个 Token ID 到 {bin_path}")
    print(f"使用的数据类型: {dtype.name}")
    print("-" * 30)

wikipedia_cn_file_path = 'F:/BaiduNetdiskDownload/data/pretrain_data/wikipedia-cn-20230720-filtered.json'
process_wikipedia(wikipedia_cn_file_path, 'F:/BaiduNetdiskDownload/data/pretrain_data/wikipedia.bin')

词汇量大小: 32000
开始处理文件: F:/BaiduNetdiskDownload/data/pretrain_data/wikipedia-cn-20230720-filtered.json
获取文件总行数...
文件总行数: 254547


Processing lines: 100%|██████████| 254547/254547 [03:22<00:00, 1258.61it/s]


写入最后 185891 个 tokens。
------------------------------
处理完成!
总共写入 121185891 个 Token ID 到 F:/BaiduNetdiskDownload/data/pretrain_data/wikipedia.bin
使用的数据类型: uint16
------------------------------


In [17]:
count_unk_token_chunked('F:/BaiduNetdiskDownload/data/pretrain_data/wikipedia.bin')

token_ids_example = np.fromfile('F:/BaiduNetdiskDownload/data/pretrain_data/wikipedia.bin', dtype=np.uint16, count=200)
print(tokenizer.decode(token_ids_example.tolist()))

文件路径: F:/BaiduNetdiskDownload/data/pretrain_data/wikipedia.bin
总数据项数: 121,185,891
数据类型: <class 'numpy.uint16'>
查找的 Token ID: 0
分块大小（项）: 52428800


处理进度: 100%|██████████| 121185891/121185891 [00:00<00:00, 352551043.78item/s]


处理完成。
总共找到的 UNK token 数量: 0
总共处理的数据项: 121185891
<s> 昭通机场（ZPZT）是位于中国云南昭通的民用机场，始建于1935年，1960年3月开通往返航班“昆明－昭通”，原来属军民合用机场。1986年机场停止使用。1991年11月扩建，于1994年2月恢复通航。是西南地区「文明机场」，通航城市昆明。 机场占地1957亩，飞行区等级为4C，有一条跑道，长2720米，宽48米，可供波音737及以下机型起降。机坪面积6600平方米，停机位2个，航站楼面积1900平方米。位于城东6公里处，民航路与金鹰大道交叉处。
航点
客服电话
昭通机场客服电话：0870-2830004</s><s> 我的英雄学院：英雄新世纪
《我的英雄学院剧场版：英雄新世纪》（仆のヒーローアカデミア THE MOVIE ヒーローズ:ライジング


### 3. 百度百科数据示例

In [19]:
import json

filepath = 'F:/BaiduNetdiskDownload/data/pretrain_data/563w_baidubaike.json'

with open(filepath, 'r', encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        print(data)
        break


{'title': '红色食品', 'summary': '红色食品是指食品为红色、橙红色或棕红色的食品。科学家认为，多吃些红色食品可预防感冒。红色食品有红柿椒、西红柿、胡萝卜、红心白薯、红果（山楂）、红苹果、草莓、红枣、老南瓜、红米、柿子等。 有治疗缺铁性贫血和缓解疲劳的作用，对乳腺癌等肿瘤疾病有防治作用，给人以兴奋感，有增加食欲，光洁皮肤，增强表皮细胞再生和防止皮肤衰老，预防感冒等作用。', 'sections': [{'title': '简介', 'content': '红色食品富含番茄红素、胡萝卜素、铁和部分氨基酸，是优质蛋白质、碳水化合物、膳食纤维、B族维生素和多种无机盐的重要来源，可以弥补粳米、白面中的营养缺失。经常食用红色食品，可以进一步提高对主食中营养的利用率，山植等食品还有治疗癌症的功效。被称为“红色生力军。”营养学家认为，红色蔬果最典型的优势在于它们都是富含天然铁质的食物，例如我们常吃的樱桃、大枣等都是贫血患者的天然良药，也适合女性经期失血后的滋补。所以，红色蔬果，女人尽可放心多吃。红色食品中还含有致病微生物的“杀手”——巨噬细胞，可以有效地抵御感冒病毒等微生物，增强人体抵抗感冒的能力。\xa0\xa0红色食品\n在所有的果蔬当中，名声最好的莫过于苹果。西方有“One apple a day，keeps the doctors away．”的说法，因为苹果性情温和，含有各种维生素和微量元素，是所有的水果中最接近完美的一个。\n还有一种说法：红色食品是相对于绿色食品而言的，指对人体有害的食品，如各种有毒有害、腐败变质、添加非食用物质的食品。红色食品危害人体健康乃至生命安全，对人体健康亮起了红灯，应当大力查处。'}, {'title': '作用', 'content': '这些食品中富含β-胡萝卜素和维生素A，对孩子上皮组织和呼吸道粘膜有很强的保护作用，可提高预防感冒的能力。\n假如你生来体质较弱，易受感冒病毒的困扰，或者已经被感冒缠上了，红色食品会助你一臂之力，天生具有促进人体健康卫士之一的巨噬细胞活力的功能，巨噬细胞乃是感冒病毒等致病微生物的“杀手”，其活力增强了，感冒病毒自然难以在人体内立足，更谈不上生长繁殖了。至于颜色较辣椒稍浅一些的胡萝卜，所含的胡萝卜素可在体内转化为维生素A，发挥护卫人体上皮组织如呼吸道黏膜的作用，常食之同样可以增强人体抗

In [21]:
import json
import os

def extract_and_filter_text(input_filepath, output_filepath, min_length=30):
    """
    Reads a large JSONL file, extracts 'summary' and 'content' from 'sections',
    filters out null or short texts, and saves them to a new JSONL file.

    Args:
        input_filepath (str): Path to the input JSONL file.
        output_filepath (str): Path to save the output JSONL file.
        min_length (int): Minimum character length for text to be included.
    """
    print(f"Starting extraction from: {input_filepath}")
    print(f"Saving filtered text to: {output_filepath}")
    print(f"Minimum text length: {min_length} characters")

    processed_lines = 0
    extracted_count = 0

    try:
        # Open input and output files simultaneously
        with open(input_filepath, 'r', encoding='utf-8') as infile, \
             open(output_filepath, 'w', encoding='utf-8') as outfile:

            for line in infile:
                processed_lines += 1
                try:
                    # Load the JSON data from the current line
                    data = json.loads(line)

                    texts_to_check = []

                    # 1. Check the 'summary' field
                    summary = data.get('summary') # Use .get() to avoid KeyError if 'summary' is missing
                    if summary: # Check if summary is not None or empty string
                        texts_to_check.append(summary)

                    # 2. Check the 'content' field within each section
                    sections = data.get('sections') # Use .get() for safety
                    if isinstance(sections, list): # Ensure 'sections' exists and is a list
                        for section in sections:
                            content = section.get('content') # Use .get() for safety
                            if content: # Check if content is not None or empty string
                                texts_to_check.append(content)

                    # 3. Filter and write qualifying texts
                    for text in texts_to_check:
                        # Ensure it's a string and meets the length requirement
                        if isinstance(text, str) and len(text.strip()) >= min_length:
                             # Create the output dictionary
                             output_record = {'text': text.strip()} # Strip leading/trailing whitespace

                             # Write the record as a JSON string to the output file, followed by a newline
                             outfile.write(json.dumps(output_record, ensure_ascii=False) + '\n')
                             extracted_count += 1

                except json.JSONDecodeError:
                    print(f"Warning: Skipping malformed JSON line {processed_lines}: {line.strip()[:100]}...") # Log malformed lines
                except Exception as e:
                    print(f"Warning: An unexpected error occurred processing line {processed_lines}: {e}") # Log other errors
                    print(f"Problematic line data (first 100 chars): {line.strip()[:100]}...")

                # Optional: Print progress periodically for large files
                if processed_lines % 100000 == 0:
                    print(f"Processed {processed_lines} lines, extracted {extracted_count} texts...")

        print("\nFinished processing.")
        print(f"Total lines processed: {processed_lines}")
        print(f"Total texts extracted and saved: {extracted_count}")
        print(f"Output file saved successfully: {output_filepath}")

    except FileNotFoundError:
        print(f"Error: Input file not found at {input_filepath}")
    except IOError as e:
        print(f"Error: Could not read/write file. Check permissions or disk space. Details: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# --- Configuration ---
input_filepath = 'F:/BaiduNetdiskDownload/data/pretrain_data/563w_baidubaike.json'
output_filepath = 'F:/BaiduNetdiskDownload/data/pretrain_data/extracted_baidubaike.jsonl'
min_char_length = 30

# --- Run the extraction ---
extract_and_filter_text(input_filepath, output_filepath, min_char_length)

Starting extraction from: F:/BaiduNetdiskDownload/data/pretrain_data/563w_baidubaike.json
Saving filtered text to: F:/BaiduNetdiskDownload/data/pretrain_data/extracted_baidubaike.jsonl
Minimum text length: 30 characters
Processed 100000 lines, extracted 255287 texts...
Processed 200000 lines, extracted 512387 texts...
Processed 300000 lines, extracted 768716 texts...
Processed 400000 lines, extracted 1021070 texts...
Processed 500000 lines, extracted 1279063 texts...
Processed 600000 lines, extracted 1534113 texts...
Processed 700000 lines, extracted 1791387 texts...
Processed 800000 lines, extracted 2046983 texts...
Processed 900000 lines, extracted 2302563 texts...
Processed 1000000 lines, extracted 2553785 texts...
Processed 1100000 lines, extracted 2807269 texts...
Processed 1200000 lines, extracted 3055883 texts...
Processed 1300000 lines, extracted 3307424 texts...
Processed 1400000 lines, extracted 3558650 texts...
Processed 1500000 lines, extracted 3810628 texts...
Processed 16

### 4. 处理sft数据集

In [6]:
import os
import json

# 查看数据集
root_path = 'F:/BaiduNetdiskDownload/data/sft_data'
file_name = 'sft_data_zh.jsonl'
file_path = os.path.join(root_path, file_name)

with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        print(data, '\n')
        messages = [
                {"role": "system", "content": "<s>system\n你是一个AI大语言模型助手，善于解答各种问题。</s>\n"},
                {"role": "user", "content": f'{data['input']}'},
                {"role": "assistant", "content": f"{data['output']}"}
            ]
        input_ids = tokenizer.apply_chat_template(messages, tokenize=False)
        print(input_ids)
        break


{'id': 1, 'instruction': '', 'input': '好的。现在请你将这个文本中的所有的逗号都替换成空格。', 'output': '好的，请稍等一下，现在我会将文本中的所有逗号替换为空格。处理后文本为："这是一个句子 目的是看看是否可以正确地从这个句子中删除关键词。"。处理结果如何？', 'history': [['给定一段文本和关键词列表，删除文本中包含所有给定关键词的子字符串。\n文本："这是一个测试句子，目的是看看模型是否可以正确地从这个句子中删除关键词。"\\n关键词列表：[‘测试’，‘模型’]', '删除包含所有给定关键词的子字符串后，文本变为："这是一个句子，目的是看看是否可以正确地从这个句子中删除关键词。"']], 'language': 'chinese', 'data_source': 'https://huggingface.co/datasets/BelleGroup/train_3.5M_CN', 'input_len': 59, 'output_len': 66, 'num_utter': 2, 'type': 31, 'type_keyword': ['字符串', '代码', '函数', '编写', '实现', '给定', '使用', '输入', '文本', '程序']} 

<s>system
你是一个AI大语言模型助手，善于解答各种问题。</s>
<s>user
好的。现在请你将这个文本中的所有的逗号都替换成空格。</s>
<s>assistant
好的，请稍等一下，现在我会将文本中的所有逗号替换为空格。处理后文本为："这是一个句子 目的是看看是否可以正确地从这个句子中删除关键词。"。处理结果如何？</s>



In [7]:
from tqdm import tqdm  # 导入 tqdm
import re
import pandas as pd
import jsonlines
import csv
import os


def chinese_ratio(text: str) -> float:
    """计算文本中中文字符的比例。"""
    if not text:
        return 0
    chinese_chars = re.findall(r'[\u4e00-\u9fff]', text)
    return len(chinese_chars) / len(text)

# 根据是否需要微调历史对话选择contain_history是否为True
def process_deepctrl(jsonl_path: str, csv_path: str, contain_history: bool = False, chunk_size: int = 1000):
    """
    处理 SFT 数据，提取输入、输出和可选的历史对话，并写入 CSV

    Args:
        jsonl_path (str): 输入 JSON Lines 文件路径。
        csv_path (str): 输出 CSV 文件路径。
        contain_history (bool, optional): 是否包含历史对话。默认为 False。
        chunk_size (int, optional): 每次写入 CSV 的数据块大小。默认为 1000。
    """
    # 如果包含历史，修改输出文件名
    if contain_history:
        dir_name, file_name = os.path.split(csv_path)
        name, ext = os.path.splitext(file_name)
        new_file_name = name + "_history" + ext
        csv_path = os.path.join(dir_name, new_file_name)

    # 准备写入 CSV
    header = ['history', 'q', 'a']
    try:
        # 尝试获取文件总行数以提供准确的进度条
        total_lines = sum(1 for _ in open(jsonl_path, 'rb')) # 使用 'rb' 更快地计数
    except Exception:
        total_lines = None # 如果文件太大或无法读取，则不显示总数

    # 初始化 CSV 文件并写入表头
    pd.DataFrame(columns=header).to_csv(
        csv_path, index=False, lineterminator='\n', quoting=csv.QUOTE_MINIMAL
    )

    processed_count = 0
    valid_chunk_data = []

    try:
        with jsonlines.open(jsonl_path) as reader, \
             tqdm(total=total_lines, desc="Processing lines", unit=" lines") as pbar:

            for idx, obj in enumerate(reader):
                pbar.update(1) # 每次读取一行就更新进度条
                try:
                    # 1. 提取和合并数据 (确保 q/a 至少有其一或 input/output)
                    q = obj.get('input', '') + obj.get('q', '')
                    a = obj.get('output', '') + obj.get('a', '')
                    history_raw = obj.get('history', []) # 默认空列表更安全

                    # 确保 history 是列表，以防万一格式不规范 (如为 null 或字符串)
                    if not isinstance(history_raw, list):
                        history_raw = []

                    # 2. 基本过滤：检查必须字段
                    if not q or not a:
                        continue
                    if contain_history and not history_raw: # 如果需要历史但历史为空，跳过
                        continue

                    # 3. 长度过滤
                    if len(q) < 5 or len(a) < 5:
                        continue

                    history_len = 0
                    if contain_history and history_raw:
                        # 确保 history 内部结构是 [q, a] 对
                        if all(isinstance(item, list) and len(item) == 2 for item in history_raw):
                            history_len = sum(len(h_q) + len(h_a) for h_q, h_a in history_raw)
                        else:
                            # 如果 history 内部结构不规范，可以选择跳过或记录日志
                            # print(f"Skipping line {idx+1}: Invalid history structure.")
                            continue # 跳过此行
                    
                     # 总字符长度过滤，同时确保没有过长的q或a，使过滤出的数据是短问题，长回答
                    if history_len + len(q) + len(a) > 450 or len(q)>50 or len(a)>400:
                        continue

                    # 4. 语言过滤 (高中文比例)
                    if not (chinese_ratio(q) > 0.9 and chinese_ratio(a) > 0.9):
                        continue

                    # 5. 构建有效记录
                    valid_record = {
                        'history': history_raw if contain_history else [],
                        'q': q,
                        'a': a
                    }
                    valid_chunk_data.append(valid_record)
                    processed_count += 1

                    # 6. 分块写入 CSV
                    if len(valid_chunk_data) >= chunk_size:
                        df_chunk = pd.DataFrame(valid_chunk_data)
                        df_chunk.to_csv(
                            csv_path, mode='a', header=False, index=False,
                            lineterminator='\n', quoting=csv.QUOTE_MINIMAL
                        )
                        valid_chunk_data = [] # 清空块

                except jsonlines.InvalidLineError:
                    print(f"\nSkipping invalid JSON line {idx + 1}")
                    continue
                except Exception as e: # 捕捉其他潜在错误
                    print(f"\nError processing line {idx + 1}: {e}")
                    continue

            # 处理并写入最后一个不足 chunk_size 的块
            if valid_chunk_data:
                df_chunk = pd.DataFrame(valid_chunk_data)
                df_chunk.to_csv(
                    csv_path, mode='a', header=False, index=False,
                    lineterminator='\n', quoting=csv.QUOTE_MINIMAL
                )

    except FileNotFoundError:
        print(f"错误：输入文件 {jsonl_path} 未找到。")
        return
    except Exception as e:
        print(f"处理过程中发生未预料的错误: {e}")
        return

    print(f"\n数据处理完成！共处理并写入 {processed_count} 条有效数据到 {csv_path}")


jsonl_path = "F:/BaiduNetdiskDownload/data/sft_data/sft_data_zh.jsonl"
csv_path = "F:/BaiduNetdiskDownload/data/sft_data/sft_data_zh.csv"
process_deepctrl(jsonl_path, csv_path)

Processing lines: 100%|██████████| 11381621/11381621 [02:39<00:00, 71408.02 lines/s] 


数据处理完成！共处理并写入 737783 条有效数据到 F:/BaiduNetdiskDownload/data/sft_data/sft_data_zh.csv


In [8]:
import pandas as pd

df = pd.read_csv(csv_path)
df.head()

,history,q,a
0,[],请从这篇文章中提取出关于分类垃圾的解决方案。,文章中提到的第二个方面——分类垃圾，是解决垃圾污染的有效措施之一。垃圾分类是指将生活垃圾按照...
1,[],根据以下文本，对此事件进行分类：中国队在足球比赛中赢得了冠军。,这个事件可以被分类为体育比赛。具体地，中国队在足球比赛中致胜并赢得了冠军。由此可以推断出，这...
2,[],回答问题并给出详细的推理过程：为什么在地球上的物体会受到重力而不会飘起来？,重力是地球的吸引力。根据牛顿定律，物体会保持以恒定的速度直线运动，直到有另一个力使它改变方向...
3,[],请告诉我什么是机器学习。,机器学习是一种人工智能技术，它使用算法和数学模型，让计算机自动地从数据中学习，并根据这些数据...
4,[],请解释一下什么是人工智能。,人工智能是一种模拟人类智能的技术和方法。它的发展包括机器学习、自然语言处理、计算机视觉等技术...


In [ ]:
index = 1
sample = df.iloc[index]
question = sample['q']
answer = sample['a']
print(question)
print(answer)
print(len(df))

messages = [
    {"role": "system", "content": "<s>system\n你是一个AI大语言模型助手，善于解答各种问题。</s>\n"},
    {"role": "user", "content": f'{question}'},
    {"role": "assistant", "content": f"{answer}"}
    ]
input_ids = tokenizer.apply_chat_template(messages, tokenize=False)
print(input_ids)

根据以下文本，对此事件进行分类：中国队在足球比赛中赢得了冠军。
这个事件可以被分类为体育比赛。具体地，中国队在足球比赛中致胜并赢得了冠军。由此可以推断出，这场比赛可能是由各国派出的足球队伍参加的国际足球比赛，中国队在比赛中表现出色并赢得了锦标。 这个事件的重要性对于中国足球运动员和球迷以及全国范围的足球运动发展都具有重要意义。
910853
<s>system
你是一个AI大语言模型助手，善于解答各种问题。</s>
<s>user
根据以下文本，对此事件进行分类：中国队在足球比赛中赢得了冠军。</s>
<s>assistant
这个事件可以被分类为体育比赛。具体地，中国队在足球比赛中致胜并赢得了冠军。由此可以推断出，这场比赛可能是由各国派出的足球队伍参加的国际足球比赛，中国队在比赛中表现出色并赢得了锦标。 这个事件的重要性对于中国足球运动员和球迷以及全国范围的足球运动发展都具有重要意义。</s>



# 三、查看模型配置

In [1]:
from utils.little_tools import load_yaml
import json

root_path = "C:/Users/WKQ/Downloads/pretrained_model/"
model_name = "mini_deepseekv3"
yaml_path = root_path + model_name + "/pretrained_mini_deepseekv3_model_args.yaml"

args = load_yaml(yaml_path)

print(json.dumps(args, indent=4))

{
    "betas": [
        0.9,
        0.95
    ],
    "bias_update_speed": 0.001,
    "dim": 768,
    "epochs": 1,
    "inter_dim": 3072,
    "kv_lora_rank": 256,
    "lr_decay_iters": 461312,
    "lr_decay_ratio": 0.98,
    "max_batch_size": 18,
    "max_lr": 0.0006,
    "max_new_tokens": 512,
    "max_seq_len": 512,
    "min_lr": 2e-05,
    "moe_inter_dim": 512,
    "mtp_loss_lambda": 0.0001,
    "n_activated_experts": 2,
    "n_dense_layers": 3,
    "n_expert_groups": 4,
    "n_heads": 12,
    "n_layers": 12,
    "n_limited_groups": 2,
    "n_routed_experts": 8,
    "n_shared_experts": 1,
    "q_lora_rank": 384,
    "qk_nope_head_dim": 64,
    "qk_rope_head_dim": 32,
    "rope_theta": 10000.0,
    "route_scale": 1.0,
    "seq_aux_alpha": 0.0001,
    "temperature": 1.0,
    "top_k": null,
    "top_p": null,
    "use_mtp": true,
    "use_noaux_tc": true,
    "use_seq_aux": true,
    "v_head_dim": 64,
    "vocab_size": 32000,
    "warmup_iters": 23536,
    "warmup_ratio": 0.05,
    "we